In [1]:
import pandas as pd 

In [2]:
FOLDER = 'data/'

## Excercise 1

In [3]:
import requests
from bs4 import BeautifulSoup as bs

In [4]:
url = 'https://bepi.mpob.gov.my/index.php/en/import/import-2021/monthly-import-of-oil-palm-products-2021'
r = requests.get(url)

print('Response status code: {0}\n'.format(r.status_code))
print('Response headers: {0}\n'.format(r.headers))

Response status code: 200

Response headers: {'Server': 'Assalamualaikum', 'Date': 'Mon, 10 May 2021 09:05:11 GMT', 'Content-Type': 'text/html; charset=utf-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Set-Cookie': 'a84e221fe625c28a19c2ec33e5dfa264=nqb2eak9iul259i6ft621asbnn; path=/; HttpOnly', 'P3P': 'CP="NOI ADM DEV PSAi COM NAV OUR OTRo STP IND DEM"', 'Expires': 'Wed, 17 Aug 2005 00:00:00 GMT', 'Last-Modified': 'Mon, 10 May 2021 09:05:11 GMT', 'Cache-Control': 'no-store, no-cache, must-revalidate, post-check=0, pre-check=0', 'Pragma': 'no-cache', 'Vary': 'Accept-Encoding', 'Content-Encoding': 'gzip'}



In [5]:
soup = bs(r.text, 'html.parser')
iframe = soup.find('iframe', class_='wrapper') #this is where a second html file, containing the desired table, is embedded
print(iframe.attrs['src'])

/../stat/web_report1.php?val=202150


In [6]:
"""
 This is a manual manipulation because urllib.urlopen wouldn't work. Also I've never seen urllib.urlopen before. So I went for the manual solution to gain time.
"""
url = 'https://bepi.mpob.gov.my/stat/web_report1.php?val=202150'
r = requests.get(url)

print('Response status code: {0}\n'.format(r.status_code))
print('Response headers: {0}\n'.format(r.headers))

Response status code: 200

Response headers: {'Server': 'Assalamualaikum', 'Date': 'Mon, 10 May 2021 09:05:12 GMT', 'Content-Type': 'text/html; charset=UTF-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Vary': 'Accept-Encoding', 'Content-Encoding': 'gzip'}



In [7]:
soup = bs(r.text, 'html.parser')
table = soup.find('table')

In [8]:
dfs = pd.read_html(str(table)) #list containing one dataframe
dfs[0].head()

,0,1,2,3,4,5,6
0,Months,Palm Oil,Palm Oil,Palm Kernel Oil,Palm Kernel Oil,Palm Kernel,Palm Kernel
1,Months,2020,2021,2020,2021,2020,2021
2,JANUARY,85033,168717,28684,30259,9442,1731
3,FEBRUARY,66735,87326,22320,21256,3803,4056
4,MARCH,79216,137332,20439,10009,5656,4147


>I stop the extraction at this stage and keep the cleaning (eliminating row 1 for example) to the next step of the processing.

In [9]:
#save into a csv file
outfile = 'html_bepi.csv'
dfs[0].to_csv(FOLDER+outfile)

## Excercise 2

In [10]:
filename = 'table2.xlsx'
data = pd.read_excel(FOLDER+filename, sheet_name=None, skiprows=3)#dict{sheet1: table1, sheet2: table2, ...}

In [11]:
"""
    The function takes a dataframe and returns a clean version of it.
"""
def clean_df(df):
    df = df.set_index('Period')
    df = df.dropna(how='all')
    df = df.dropna(axis='columns', how='all')
    return df

In [12]:
clean_dfs = []
for key in data:
        clean_dfs.append(clean_df(data[key]))

In [13]:
result = pd.concat(clean_dfs, axis=1)
result.head()

,Corn,Grain Sorghum,Agriculture and forestry residues,Dedicated energy crops,Other,Poultry,Tallow (beef),White Grease,Yellow Grease,Other,...,Corn oil,Palm oil,Sorghum oil,Soybean oil,Other vegetable oils,Municipal solid waste,Yard and food waste,Other,Biogas,Other biofuel feedstocks (NESOI)
Period,,,,,,,,,,,,,,,,,,,,,
January,23237.114,W,0.0,0.0,0.0,33.535,83.999,52.546,262.722,5.586,...,213.334,0.0,0.0,682.876,0.0,0.0,W,63.527000,0.0,19.834
February,18716.806,W,0.0,0.0,0.0,34.245,66.431,50.412,197.612,3.393,...,155.020,0.0,0.0,552.228,0.0,0.0,W,51.749381,0.0,10.503


>It would probably be judicious to make the dataframe vertical and create a multiindex (categroy, subcategory) because we have column 'Other' multiple times and that is confusing. Other solution would be to simply rename the columns 'Other'.

In [14]:
#save into a csv file
outfile = 'xlsx_us_feedstocks.csv'
result.to_csv(FOLDER+outfile)

## Exercise 3

In [15]:
%pip install tabula-py

Note: you may need to restart the kernel to use updated packages.


In [16]:
from tabula import read_pdf

In [17]:
filename = 'Area_summary.pdf'
dfs = read_pdf(FOLDER+filename, pages='all')
dfs[0]

,Unnamed: 0,Unnamed: 1,MATURE,Unnamed: 2,IMMATURE,Unnamed: 3,TOTAL,Unnamed: 4
0,NaN,STATE,NaN,%,NaN,%,NaN,%
1,NaN,NaN,(HA),NaN,(HA),NaN,(HA),NaN
2,JOHOR,NaN,"688,291",92.9,"52,537",7.1,"740,828",12.6
3,KEDAH,NaN,"80,210",89.3,"9,572",10.7,"89,782",1.5
4,KELANTAN,NaN,"131,768",78.6,"35,831",21.4,"167,599",2.9
5,MELAKA,NaN,"51,672",91.7,"4,689",8.3,"56,361",1.0
6,NEGERI SEMBILAN,NaN,"173,490",91.1,"16,972",8.9,"190,462",3.2
7,PAHANG,NaN,"702,163",89.8,"80,084",10.2,"782,247",13.3
8,PERAK,NaN,"352,557",90.0,"39,211",10.0,"391,768",6.7
9,PERLIS,NaN,689,99.3,5,0.7,694,0.0


>Again, I think that cleaning the dataframe should be kept to the next step of the processing. Here are only minor manual manipulations that are needed to delete the extra columns (column 'Unnamed:1') and rows (0 and 1) caused by the fusioned cells on the original table.

In [18]:
#save into a csv file
outfile = 'pdf_area_summary.csv'
dfs[0].to_csv(FOLDER+outfile)